# TF-IDF를 활용한 모델 구현

In [0]:
import os

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [0]:
DATA_IN_PATH = r'./data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

# TF-IDF 벡터화

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

# min_df : 설정한 값보다 특정 토큰의 df값이 더 적게 나오면 벡터화 과정에서 제거
# analyzer : 분석하기 위한 기준 단위. 'word'는 단어 하나 단위, 'char'는 문자 하나 단위
# sublinear_tf : 문서의 단어 빈도 수(term frequency)에 대한 스무딩 여부 설정
# ngram_range : 빈도의 기본 단위를 어느 범위의 n-gram으로 설정할 것인지의 값
# max_features : 각 벡터의 최대 길이, 특징의 길이 설정
vectorizer = TfidfVectorizer(min_df=0.0, analyzer='char', sublinear_tf=True,
                             ngram_range=(1, 3), max_features=5000)

X = vectorizer.fit_transform(reviews)

# 학습과 검증 데이터셋 분리

In [0]:
RANDOM_SEED = 42
TEST_SPLIT = 0.2

y = np.array(sentiments)

X_train, X_eval, y_train, y_eval =  train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

# 모델 선언 및 학습

In [0]:
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

# 검증 데이터로 성능 평가

In [0]:
print("Accuracy: %f" % model.score(X_eval, y_eval))

Accuracy: 0.859600


# 캐글에 데이터 제출

In [0]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

In [0]:
testDataVecs = vectorizer.transform(test_data['review'])

In [0]:
test_predicted = model.predict(testDataVecs)
print(test_predicted)

[1 0 1 ... 0 1 0]


In [0]:
DATA_OUT_PATH = r'./data_out/'

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

ids = list(test_data['id'])
answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH+'lgs_tfidf_answer.csv', index=False, quoting=3)